In [227]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
from flask import Flask, request, jsonify

In [228]:
# Creación del dataset

df = pd.read_csv('data/Chronic_Kidney_Dsease_data.csv')
df.drop(columns=['PatientID', 'DoctorInCharge'], inplace=True)
df = pd.get_dummies(df, columns=['Ethnicity', 'SocioeconomicStatus', 'EducationLevel'])

# Comenzamos el modelo
X = df.drop(columns=['Diagnosis'])
y = df['Diagnosis']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1453)

xgb_cls = xgb.XGBClassifier(random_state = 1453)


# Declaramos algunos parametros
xgb_cls.set_params(n_stimators=10, max_depth= 3)
xgb_cls.fit(X_train, y_train)


from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

# Comprobamos datos del train
print(confusion_matrix(y_train, xgb_cls.predict(X_train)))
print(accuracy_score(y_train, xgb_cls.predict(X_train)))
print(recall_score(y_train, xgb_cls.predict(X_train)))
print(precision_score(y_train, xgb_cls.predict(X_train)))

# Comprobamos datos del test
print(confusion_matrix(y_test, xgb_cls.predict(X_test)))
print(accuracy_score(y_test, xgb_cls.predict(X_test)))
print(recall_score(y_test, xgb_cls.predict(X_test)))
print(precision_score(y_test, xgb_cls.predict(X_test)))

[[ 108    0]
 [   0 1219]]
1.0
1.0
1.0
[[  9  18]
 [  0 305]]
0.9457831325301205
1.0
0.9442724458204335


c:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [20:46:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_stimators" } are not used.

  warnings.warn(smsg, UserWarning)


In [229]:
# Guardar en un pickle el modelo y los train y test
with open('model_and_test_train.pkl', 'wb') as file:
    pickle.dump((df, xgb_cls, X_train, X_test, y_train, y_test), file)




In [230]:
# Cargar el modelo del pickle
with open('model_and_test_train.pkl', 'rb') as file:
    df, xgb_cls, X_train, X_test, y_train, y_test = pickle.load(file)

Pasar nuestros datos a JSON

In [231]:
df_json = df.to_json(orient='records')
with open('data/Chronic_Kidney_Disease_data.json', 'w') as file:
    file.write(df_json)

In [232]:
app = Flask(__name__)
app.config['DEBUG'] = False

In [233]:
@app.route('/', methods=['GET'])
def home():
	return "<h1>Estudio de enfermedades renales crónicas</h1><p>Esta API nos aporta los datos de unos pacientes sobre los que se ha estudiado enfermedades renales crónicas.</p>"


# TODOS LOS DATOS
@app.route('/api/v1/resources/data/all', methods=['GET']) 
def api_all():

#    df = pd.read_csv('data/Chronic_Kidney_Dsease_data.csv')
    df_json = df.to_json(orient='records')
    return df_json

# TEST

@app.route('/api/v1/resources/data/test', methods=['GET'])
def api_test():
    y_test_join = pd.DataFrame(y_test)
    y_test_reset = y_test_join.reset_index()
    y_test_reset

    X_test_join = pd.DataFrame(X_test)
    X_test_reset = X_test_join.reset_index()
    X_test_reset

    test = X_test_reset.merge(y_test_reset, on='index')
    test = test.drop(columns=['index'])
    test_json = test.to_json(orient='records')
    return test_json


# TRAIN

@app.route('/api/v1/resources/data/train', methods=['GET'])
def api_train():
    y_train_join = pd.DataFrame(y_train)
    y_train_reset = y_train_join.reset_index()
    y_train_reset

    X_train_join = pd.DataFrame(X_train)
    X_train_reset = X_train_join.reset_index()
    X_train_reset

    train = X_train_reset.merge(y_train_reset, on='index')
    train = train.drop(columns=['index'])
    train_json = train.to_json(orient='records')
    return train_json


# No he conseguido que esto termine de funcionar
#@app.route('/api/v1/resources/data/all', methods=['GET']) 
#def api_observacion():
#        df_index = df.reset_index()
#        if 'index' in request.args: 
#            index_ = int(request.args['index'])
#            results = df_index[df_index['index'] == index_]
#            results_json = results.to_json(orient='records')
#            return results_json


# OBSERVACION N DEL DATASET

@app.route('/api/v1/resources/data/all/<int:index>', methods=['GET'])
def api_observacion(index):
    df_index = df.reset_index()
    results = df_index[df_index['index'] == index]
    
    if not results.empty: 
        results_json = results.to_json(orient='records')
        return results_json
    
    else:
        return jsonify('No se ha encontrado ningún paciente con ese index')
         

# QUERY CON LA QUE OBTENEMOS UNA ORQUILLA DE EDADES
@app.route('/api/v1/resources/data/query', methods=['GET']) 
def api_query():

    from_ = int(request.args["from"])
    to_ =  int(request.args["to"])
    results_query = df[(df["Age"]>=from_) & (df["Age"]<=to_)]
    results_query_json = results_query.to_json(orient='records')
    return results_query_json

# No consigo hacerlo funcionar, me da error 400 constantemente
#@app.route('/api/v1/resources/data/query', methods=['GET']) 
#def api_query_diagnosis():
#
#    diagnosis_ = int(request.args["diagnosis"])
#    results_query_diag = df[df["Diagnosis"] == diagnosis_]
#    results_query_diag = df[(df["Diagnosis"]==diagnosis_)]
#    results_query_diag_json = results_query_diag.to_json(orient='records')
#    return results_query_diag_json


# PREDICCIONES solo con test

#@app.route('/api/v1/resources/data/predictions/test', methods=['GET']) 
#def api_predictions():
#    predictions = pd.DataFrame(xgb_cls.predict(X_test), columns=['prediction'])
#    predictions['patient'] = df.loc[X_test.index].index
#    predictions = predictions.reindex(['patient', 'prediction'], axis=1)
#
#    predictions_json = predictions.to_json(orient='records')
#
#    return predictions_json


# QUERY CON LA QUE OBTENEMOS LAS PREDICCIONES

@app.route('/api/v1/resources/data/predictions', methods=['GET']) 
def api_predictions_query():

    type_ = str(request.args["type"])
    
    if type_ == 'test':
            predictions_test = pd.DataFrame(xgb_cls.predict(X_test), columns=['prediction'])
            predictions_test['patient'] = df.loc[X_test.index].index
            predictions_test = predictions_test.reindex(['patient', 'prediction'], axis=1)

            predictions_test_json = predictions_test.to_json(orient='records')

            return predictions_test_json
    
    elif type_ == 'train':
            predictions_train = pd.DataFrame(xgb_cls.predict(X_train), columns=['prediction'])
            predictions_train['patient'] = df.loc[X_train.index].index
            predictions_train = predictions_train.reindex(['patient', 'prediction'], axis=1)

            predictions_train_json = predictions_train.to_json(orient='records')

            return predictions_train_json

app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


# URL´s

In [ ]:
# http://127.0.0.1:5000/api/v1/resources/data/all

# http://127.0.0.1:5000/api/v1/resources/data/test
# http://127.0.0.1:5000/api/v1/resources/data/train

# http://127.0.0.1:5000/api/v1/resources/data/all/60

# http://127.0.0.1:5000/api/v1/resources/data/query?from=50&to=72

# http://127.0.0.1:5000/api/v1/resources/data/predictions?type=test
# http://127.0.0.1:5000/api/v1/resources/data/predictions?type=train